In [23]:
from collections import defaultdict
from copy import deepcopy
from pprint import pprint
from tqdm import tqdm

In [4]:
q = 5
F.<a> = GF(q)
d = 3

R.<x> = PolynomialRing(F)
p = F.characteristic()
r = F.degree()

In [5]:
def numRatlEquivClasses(p, q, n): 
    def A(q, n): 
        a = sum([euler_phi(d) * (q^(2*n/d-2) + (d-1)*(q^(2*n/d)-1)/(q+1)) for d in divisors(gcd(q-1, n)) if d > 1])
        b = sum([euler_phi(d) * (q^(2*floor(n/d)+1) + 1)/(q+1) for d in divisors(q-1) if (d not in divisors(n))])
        return a + b 

    def B(q, n):
        a = sum([euler_phi(d) * (q^(2*n/d-2) + (q+1) * (q^(2*n/d) - (-1)^(n/d))/(q^2 + 1)) for d in divisors(gcd((q+1), n)) if d % 2 == 0])
        b = sum([euler_phi(d) * q^(2*n/d-2) for d in divisors(gcd((q+1), n)) if (d % 2 == 1 and d > 1)])
        c = 1/((q+1)*(q^2+1)) * sum([euler_phi(d) * ((1+(-1)^(n//d))/2 * (1+q)^2 + q*(q^(2 * n//d + 2) -1)) for d in divisors(q+1) if d not in divisors(n)])
        d = 1/(q^2 + 1) * sum([euler_phi(d) * ((-1)^(n//d) * (1+q) + q^(2 * (n//d) + 1) * (q-1)) for d in divisors(q+1) if d in divisors(q+1) if d not in divisors(n)])
        # print("B terms are", a, b, c, d)
        return a + b + d

    def C(q, n):
        if n % p == 0:
            return q^(2*n/p-1) 
        elif n == 1: 
            return 1 
        elif n % p == 1: 
            return q^(2*(n-1)/p-1) * (q-1) 
        else: 
            return 0

    # print("Each components is", A(q, n), B(q, n), C(q, n))
    
    return (q^(2*n-3))/(q^2-1) + 1/(2*(q-1)) * A(q, n) + 1/(2*(q+1)) * B(q, n) + 1/q * C(q,n)

In [27]:
print(numRatlEquivClasses(5, 5, 3))

10


In [40]:
# very slow code; pls scratch this later on but I needed some data for very small case 
PGL = [(a*x + b)/(c*x + d) for a in F for b in F for c in F for d in F if a*d-b*c != 0]

def ratlEquiv(f, g):
    """returns whether or not f is PGL-equiv to g""" 
    for alpha in PGL: 
        for beta in PGL: 
            try: 
                if alpha(f(beta)) == g: 
                    return (true, alpha, beta)
            except ZeroDivisionError: 
                continue
    return false

deg3ratls = set(f / (x^3 + g) for f in R.polynomials(max_degree=2) for g in R.polynomials(max_degree=2) if gcd(f, x^3 + g) == 1)

def reallyslowRatlEquivClasses(print_progress=False): 
    classes = set()
    total = len(deg3ratls)
    candidates = deepcopy(deg3ratls)
    if print_progress:
        print(f"{total - len(candidates)} out of {total} done", end="\r")
    while candidates:
        f = candidates.pop()
        for alpha in PGL: 
            for beta in PGL: 
                try: 
                    candidates.discard(alpha(f(beta)))
                except ZeroDivisionError: 
                    continue
        classes.add(f)
        if print_progress:
            print(f"{total - len(candidates)} out of {total} done", end="\r")
    if print_progress:
        print()
    return classes

def valueFreq(f): 
    """given a rational function f, compute the value frequency of it"""
    frequencies = defaultdict(int)
    for a in F:
        try: 
            value = f(a)
        except ZeroDivisionError: 
            value = "infinity" 
        frequencies[value] += 1
    return sorted(frequencies.values()) 

In [41]:
classes = reallyslowRatlEquivClasses(print_progress=True)

value_freqs = []
for f in tqdm(classes):
    value_freqs.append((f, valueFreq(f)))
pprint(value_freqs)

12500 out of 12500 done


100%|██████████| 10/10 [00:00<00:00, 13644.45it/s]

[((x^2 + 4*x + 2)/(x^3 + 3*x^2 + 3*x + 4), [1, 1, 1, 2]),
 ((4*x^2 + x)/(x^3 + 3*x^2 + 3*x + 4), [1, 1, 1, 2]),
 (3/(x^3 + 4*x^2 + 2*x + 3), [1, 1, 1, 1, 1]),
 ((4*x^2 + x + 2)/(x^3 + 3*x^2 + x + 2), [1, 2, 2]),
 ((3*x^2 + 2*x)/(x^3 + 2*x^2 + x + 3), [1, 1, 1, 2]),
 ((x^2 + 4*x + 4)/(x^3 + 2*x^2 + x + 3), [1, 1, 1, 2]),
 (4*x/(x^3 + 3*x^2 + 4*x + 3), [1, 1, 1, 2]),
 (4*x/(x^3 + 2*x^2 + 4*x + 4), [1, 1, 1, 2]),
 ((4*x^2 + 2*x + 3)/(x^3 + 2*x^2 + 2*x), [2, 3]),
 ((3*x^2 + 3)/(x^3 + 2*x + 1), [1, 1, 1, 2])]
